# Meeting Note Summarizer

Before starting Jupyter, we need to set the API keys to `init_env_vars.sh` file and source it as `source init_env_vars.sh` to make the environment variables available.

In [10]:
import os

ASSEMBLYAI_APIKEY = os.getenv('ASSEMBLYAI_APIKEY')
OPENAI_APIKEY = os.getenv('OPENAI_APIKEY')

if ASSEMBLYAI_APIKEY:
    print(f"ASSEMBLY AI API key: {ASSEMBLYAI_APIKEY}")

if OPENAI_APIKEY:
    print(f"OPENAI API key: {OPENAI_APIKEY}")    